In [2]:
import pandas as pd
# %conda install  beautifulsoup4
from bs4 import BeautifulSoup
import requests
import re
# %conda install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')  # Needed the first time you use NLTK's tokenizers
from nltk.tokenize import word_tokenize
from transformers import pipeline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', 50)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kavach/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kavach/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/kavach/miniconda3/envs/tfenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
print('Hello')

Hello


In [78]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

# Function to clean the DataFrame and convert it to a list
def clean_dataframe_and_convert_to_list(df):
    def strip_html(text):
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text()

    def remove_between_square_brackets(text):
        return re.sub(r'\[[^]]*\]', '', text)

    def denoise_text(text):
        text = strip_html(text)
        text = remove_between_square_brackets(text)
        return text

    def remove_special_characters(text, remove_digits=True):
        pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
        text = re.sub(pattern, '', text)
        return text
    
    # Apply the cleaning functions
    df['Content'] = df['Content'].apply(lambda x: x.lower())
    df['Content'] = df['Content'].apply(denoise_text)
    df['Content'] = df['Content'].apply(remove_special_characters)
    
    # Convert the DataFrame into a list of dictionaries
    result_list = df.to_dict('records')
    
    return result_list

# Function to fetch reviews and return cleaned data as a list of dictionaries
def fetch_reviews_to_df(movie_id):
    api_key = '472483140ad07905a27f7ff2eed59152'
    response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}/reviews?api_key={api_key}&language=en-US")
    
    if response.status_code == 200:
        data = response.json()
        reviews = data.get('results', [])
        
        reviews_data = []
        for review in reviews[:4]:  # Limit to first 4 reviews
            review_dict = {
                "Author": review.get('author'),
                "Content": review.get('content')
            }
            reviews_data.append(review_dict)
        
        # Convert list of dictionaries to DataFrame
        df_reviews = pd.DataFrame(reviews_data)
        
        # Clean the DataFrame and convert it back to a list of dictionaries
        cleaned_reviews_list = clean_dataframe_and_convert_to_list(df_reviews)
        
        return cleaned_reviews_list
    else:
        return []



In [80]:
fetch_reviews_to_df(285)

[{'Author': 'Andre Gonzales',
  'Content': 'part  jack is stuck in davy jones locker while he is stuck the biggest war is being set to happen jack and the others try to free him from davy jones locker before the war comes very good movie as well kind of boring at times though'},
 {'Author': 'CinemaSerf',
  'Content': 'now this is just far far too long at ten minutes shy of three hours the story just isnt substantial enough to sustain it and i must confess to finding my attention dwindling a bit at various stages of the proceedings thankfully geoffrey rush has rejoined the cast as the whole ensemble must now risk life and limb  and sail to the very edge of the world  to save jack johnny depp from oblivion and thwart the seemingly unstoppable lord beckett tom hollander how to do this well they must galvanise the entire global pirate community and that means the dreaded sao feng chow yunfat capt teague the one and only keith richards and a consortium of the most corrupt venal and treacher